In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
from shapely.geometry import LineString
from linref.events.geometry import LineStringM, linemerge_m
import shapely

l1 = shapely.from_wkt('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2)')
l2 = shapely.from_wkt('LINESTRING (3 2, 4 3, 5 3, 6 4, 7 5)')
l3 = shapely.from_wkt('LINESTRING (8 5, 8 6, 9 7, 10 8, 11 9)')
lm1 = LineStringM(l1, [1, 5, 20, 30, 40])
lm2 = LineStringM(l2, [40, 60, 70, 80, 90])
lm3 = LineStringM(l3, [100, 110, 120, 130, 140])

In [3]:
l12 = shapely.ops.linemerge([l1, l2])
l123 = shapely.ops.linemerge([l1, l2, l3])
l12.wkt, l123.wkt

('LINESTRING (0 0, 1 0, 1 1, 2 1, 3 2, 4 3, 5 3, 6 4, 7 5)',
 'MULTILINESTRING ((0 0, 1 0, 1 1, 2 1, 3 2, 4 3, 5 3, 6 4, 7 5), (8 5, 8 6, 9 7, 10 8, 11 9))')

In [7]:
linemerge_m([lm2, lm1], allow_multiple=True, squeeze=False)

[LINESTRING M (0.0 0.0 1, 1.0 0.0 5, 1.0 1.0 20, 2.0 1.0 30, 3.0 2.0 40, 4.0 3.0 60, 5.0 3.0 70, 6.0 4.0 80, 7.0 5.0 90) # linref compatibility approximation]

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
df_linear_1 = pd.DataFrame(index=[7, 6, 9, 4, 3, 2, 0, 1], data={
    'county': ['A', 'A', 'A', 'A', 'A' ,'B', 'B', 'B'],
    'route': ['Main St', 'Main St', 'Main St', '1st Ave', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 0.5, 1.1, 10.0, 12.0, 13.0, 13.1, 1.8],
    'end': [0.4, 1.1, 3.0, 12.0, 13.0, 13.1, 14.0, 3.0],
    'volume': [100, 200, 100, 400, 500, 150, 200, 800],
    'class': ['B', 'A', 'A', 'A', 'B', 'A', 'A', 'A'],
})
#df_linear_1 = pd.concat([df_linear_1] * 100, ignore_index=True)#.sort_values(['route', 'beg'])
df_linear_2 = pd.DataFrame({
    'route': ['Main St', 'Main St', '1st Ave', '2nd Ave', '2nd Ave', 'Sycamore School Rd'],
    'beg': [0.0, 1.0, 10.0, 13.0, 13.1, 2.2],
    'end': [1.0, 2.0, 12.0, 13.0, 14.0, 2.8],
    'street_type': ['A', 'B', 'A', 'A', 'B', 'A'],
})
#df_linear_2 = pd.concat([df_linear_2] * 100, ignore_index=True)#.sort_values(['route', 'beg'])

ed1 = EventsData(index=df_linear_1.index, groups=df_linear_1.route, begs=df_linear_1.beg, ends=df_linear_1.end, closed='left_mod')
ed2 = EventsData(index=df_linear_2.index, groups=df_linear_2.route, begs=df_linear_2.beg, ends=df_linear_2.end, closed='left_mod')
ed3 = EventsData(index=df_linear_1.index, groups=df_linear_1.route, locs=df_linear_1.beg)

In [26]:
from linref.ext.base import LRS
lrs = LRS(['county', 'route'], 'loc', 'beg', 'end', 'geom', closed='left_mod')
df = pd.concat([df_linear_1] * 2000, ignore_index=True)
df.lr.set_lrs(lrs)

In [28]:
dfs = []
for i in range(2000):
    x = df_linear_1.copy()
    x['county'] = x['county'] + str(i)
    dfs.append(x)
df = pd.concat(dfs, ignore_index=True)
df.lr.set_lrs(lrs)

In [29]:
df.lr.dissolve(retain=['class'])

,county,route,class,beg,end,dissolved_index
0,A0,Main St,B,0.0,0.4,[0]
1,A0,Main St,A,0.5,3.0,"[1, 2]"
2,A0,1st Ave,A,10.0,12.0,[3]
3,A0,1st Ave,B,12.0,13.0,[4]
4,B0,2nd Ave,A,13.0,14.0,"[5, 6]"
...,...,...,...,...,...,...
11995,A1999,Main St,A,0.5,3.0,"[15993, 15994]"
11996,A1999,1st Ave,A,10.0,12.0,[15995]
11997,A1999,1st Ave,B,12.0,13.0,[15996]
11998,B1999,2nd Ave,A,13.0,14.0,"[15997, 15998]"


In [30]:
ev = df.lr.events
er = ev.relate(ev)

In [31]:
ev.num_events

16000

In [36]:
%%prun
er.overlay(chunksize=1000, grouped=True)

         7561959 function calls (7561954 primitive calls) in 5.806 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     8000    1.799    0.000    1.881    0.000 _arraysetops_impl.py:862(_in1d)
    16000    0.366    0.000    2.239    0.000 _construct.py:903(_block)
    80008    0.298    0.000    0.439    0.000 _sputils.py:149(get_index_dtype)
    40001    0.194    0.000    0.917    0.000 _coo.py:185(_check)
   176009    0.177    0.000    0.177    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    40001    0.167    0.000    1.675    0.000 _coo.py:27(__init__)
     8000    0.155    0.000    0.356    0.000 relate.py:251(overlay)
    32001    0.121    0.000    0.260    0.000 selection.py:124(_apply_selector)
   232011    0.102    0.000    0.199    0.000 {built-in method builtins.any}
    48003    0.101    0.000    0.262    0.000 copy.py:61(copy)
    88003    0.089    0.000    0.252    0.000 _coo.py:161(_getnnz)
   160016    0.

In [39]:
%%prun
er.intersect(chunksize=1000, grouped=True)

         7842021 function calls (7842011 primitive calls) in 5.995 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     8000    1.828    0.000    1.913    0.000 _arraysetops_impl.py:862(_in1d)
    16000    0.370    0.000    2.270    0.000 _construct.py:903(_block)
    80008    0.318    0.000    0.464    0.000 _sputils.py:149(get_index_dtype)
    40001    0.202    0.000    0.951    0.000 _coo.py:185(_check)
    40001    0.173    0.000    1.735    0.000 _coo.py:27(__init__)
     8000    0.168    0.000    0.417    0.000 relate.py:400(intersect_linear_linear)
   160009    0.154    0.000    0.154    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    32001    0.124    0.000    0.267    0.000 selection.py:124(_apply_selector)
    48001    0.114    0.000    0.204    0.000 _function_base_impl.py:5644(append)
   232011    0.103    0.000    0.198    0.000 {built-in method builtins.any}
    48003    0.103    0.000    0.269    0.000 c

In [34]:
import linref_011 as lr011
ec1 = lr011.EventsCollection(df, ['county', 'route'], 'beg', 'end', closed='left_mod')
ec2 = lr011.EventsCollection(df, ['county', 'route'], 'beg', 'end', closed='left_mod')
em = ec1.merge(ec2)

In [35]:
%%prun
em.build()

         16191898 function calls (15855872 primitive calls) in 9.870 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   128004    0.458    0.000    1.110    0.000 take.py:120(_take_nd_ndarray)
2782247/2782243    0.401    0.000    0.677    0.000 {built-in method builtins.isinstance}
   112004    0.371    0.000    0.386    0.000 cast.py:550(maybe_promote)
   160006    0.261    0.000    0.261    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    32000    0.213    0.000    0.351    0.000 generic.py:278(__init__)
    16000    0.205    0.000    0.792    0.000 managers.py:708(_slice_take_blocks_ax0)
     8000    0.199    0.000    0.274    0.000 collection.py:1473(intersecting)
    32004    0.196    0.000    0.303    0.000 cast.py:1156(maybe_infer_to_datetimelike)
1208076/872068    0.176    0.000    0.237    0.000 {built-in method builtins.len}
    32004    0.171    0.000    1.206    0.000 base.py:475(__new__)
    48004    0.157 